In [595]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from sklearn.feature_extraction.text import CountVectorizer
stop = stopwords.words('english')

filepath='trainset.txt'   #file_path
names=['class','title','date','body']   # headers
dataset = pd.read_csv(filepath,delimiter="\t", names = names) # read the file 
#print('Shape o data : ', dataset.shape)
#print(dataset.describe())
labels=dataset['class']
dataset=dataset.drop(['date','class'], axis=1)

dataset['body']=dataset['body'].str.lower()
dataset['title']=dataset['title'].str.lower()
body=dataset['body']
title=dataset['title']

import re
from nltk.stem import PorterStemmer 
regex = re.compile('[^a-zA-Z]')
ps = PorterStemmer() 


def preprocessor(dataset):
    for i in range(len(dataset['body'])-1):
        dataset['body'][i]=regex.sub(' ',(dataset['body'][i]))
        dataset['body'][i]=word_tokenize(dataset['body'][i])
        temp_words=""
        for w in dataset['body'][i]:
            if w not in stop_words:
                w=ps.stem(w)
                temp_words=temp_words+" "+w
        dataset['body'][i]=temp_words
    
    for i in range(len(dataset['title'])-1):
        dataset['title'][i]=regex.sub(' ',(dataset['title'][i]))
        dataset['title'][i]=word_tokenize(dataset['title'][i])
        temp_words=""
        for w in dataset['title'][i]:
            if w not in stop_words:
                w=ps.stem(w)
                #temp_words.append(w)
                temp_words=temp_words+" "+w
        dataset['title'][i]=temp_words
    dataset['data'] = dataset[['title', 'body']].apply(lambda x: ' '.join(x), axis=1)
    dataset=dataset.drop(['body','title'], axis=1)
    return dataset

#making new column with Title and Body
dataset=preprocessor(dataset)

In [596]:
cv = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'),lowercase=True)
X = cv.fit_transform(dataset['data'])
#print(dataset['data'])
y=labels
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1) 

X_train=X_train.toarray()
X_test=X_test.toarray()

In [597]:
from sklearn.naive_bayes import GaussianNB 
gnb = GaussianNB() 
gnb.fit(X_train, y_train) 

y_pred = gnb.predict(X_test) 
#print(y_pred)
#print(y_test)
from sklearn import metrics 
print("Gaussian Naive Bayes model accuracy(in %):", metrics.accuracy_score(y_test, y_pred)*100)

Gaussian Naive Bayes model accuracy(in %): 95.0


In [598]:
filepath='testsetwithoutlabels.txt'   #file_path
names=['title','date','body']
dataset_pred = pd.read_csv(filepath,delimiter="\t", names = names) # read the file 
#print(dataset_pred)
dataset_pred=dataset_pred.drop(['date'], axis=1)
dataset_pred=preprocessor(dataset_pred)
X_pred = cv.transform(dataset_pred['data'])
#print(dataset_pred['data'])

In [599]:
print(X_pred.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]]


In [600]:
y_pred=gnb.predict(X_pred.toarray())
print(y_pred)

[ 1  1  1 -1 -1  1 -1 -1  1 -1 -1 -1  1  1 -1  1  1 -1  1  1 -1 -1 -1 -1
 -1 -1  1 -1 -1 -1 -1  1 -1  1 -1  1 -1  1 -1 -1 -1  1 -1 -1  1 -1  1 -1
  1 -1  1  1 -1 -1  1 -1  1  1  1 -1 -1 -1 -1 -1  1 -1  1  1  1 -1  1 -1
 -1 -1  1 -1  1  1  1 -1  1 -1  1 -1  1 -1 -1  1  1  1  1  1  1  1  1 -1
 -1  1 -1  1]
